# Auto Scraper


In [12]:
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import statistics
import undetected_chromedriver as uc

In [13]:
def initiate (enlace=0):
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = uc.Chrome()
    driver.implicitly_wait(5)
    
    driver.get('https://www.fotocasa.es/es/')
    time.sleep(3)
    driver.maximize_window()
    time.sleep(2)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar y cerrar')]"))).click()
    time.sleep(3)
    return driver
    

In [14]:
#driver= initiate()

In [34]:
def buscar_distrito(distrito, driver):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomInput-input sui-AtomInput-input-size-m'.replace(' ', '.')))).click()
    time.sleep(3)
    
    buscar= driver.find_element(by=By.CLASS_NAME,value="sui-AtomInput-input sui-AtomInput-input-size-m".replace(' ', '.'))
    buscar.send_keys(distrito)
    time.sleep(6)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Buscar')]"))).click()

In [16]:
#buscar_distrito('san esteban leon',driver)

In [17]:
def scroll_down(driver):  
    elem = driver.find_element(By.TAG_NAME,value="body")
    no_of_pagedowns = 25
    time.sleep(3)
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        no_of_pagedowns-=1
        
    time.sleep(3)
    element = driver.find_element(By.CLASS_NAME,'sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded'.replace(' ', '.'))
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()   

In [18]:
#scroll_down(driver)

In [19]:
def get_html(driver):
    html_txt=driver.page_source
    soup=BeautifulSoup(html_txt)
    viviendas= soup.find('section',class_='re-SearchResult')
    return viviendas

In [20]:
#viviendas=get_html(driver)

In [21]:
def get_data(viviendas,distrito):
    try:
        vivienda_str_list=[]

        for vivienda in viviendas:     
            vivienda_str_len=len(str(vivienda))
            vivienda_str_list.append(vivienda_str_len)

        mean_len=statistics.mean(vivienda_str_list)*0.80
        print('Mean length value of the html:',mean_len)
        
        precios,titulos,habitaciones,baños,superficie,ascensor,planta,link,fechas = get_vivienda_data(viviendas, mean_len)

        df=pd.DataFrame(list(zip(titulos,precios,habitaciones,planta,superficie,baños,ascensor,fechas ,link))).drop_duplicates()
        df=df.rename(columns={0:"Título",1:"Precio",2:"Habitaciones",3:"Planta",4:"Superficie",5:"Baños",6:"Ascensor",7:"Fecha Publicación",8:"Links"})
        ciudades = ['Leon', 'Alicante']
        df['Barrio']=distrito
        for ciudad in ciudades:
            if ciudad in distrito: 
                df['Ciudad']=ciudad
    except:
        df=pd.DataFrame()
    return df
    
    
def get_vivienda_data(viviendas, mean_len):   
    lista_precios=[]
    lista_titulos=[]
    lista_habitaciones=[]
    lista_baños=[]
    lista_superficie=[]
    lista_ascensor = []
    lista_planta = []
    lista_links = []
    lista_fechas = []
    
    for vivienda in viviendas:
        try:
            #data required
            vivienda_str = str(vivienda)
            hab_pos = re.search(r'habs.', vivienda_str)
            hab_pos_indi = re.search(r'hab.', vivienda_str)
    
            baño_pos = re.search(r'(baño|Baño)', vivienda_str)
    
            superficie_pos = re.search(r'([0-9] [m])', vivienda_str)
            keyword='ascensor'

            planta='Planta'
            planta_baja='planta'
            
            #precios
            precio = vivienda.find('span', class_='re-CardPrice').getText()
            lista_precios.append(precio)
            
            #titulos
            titulos = vivienda.find('span', class_='re-CardTitle').getText()
            lista_titulos.append(titulos)
            
            #habitaciones
            if hab_pos:
                hab_pos_text = hab_pos.start()
                hab = vivienda_str[hab_pos_text-2:hab_pos_text-1]
                lista_habitaciones.append(hab)
            elif hab_pos_indi:
                hab_pos_text = hab_pos_indi.start()
                hab = vivienda_str[hab_pos_text-2:hab_pos_text-1]
                lista_habitaciones.append(hab)       
            else:
                lista_habitaciones.append(None)
            
            
            #baños
            if baño_pos:
                baño_pos_text = baño_pos.start()
                b = vivienda_str[baño_pos_text-2:baño_pos_text-1]
                lista_baños.append(b)
            else:
                lista_baños.append(None)
            
            
            #superficie
            if superficie_pos:
                superficie_pos_text = superficie_pos.start()
                s = vivienda_str[superficie_pos_text-2:superficie_pos_text+3]

                if 'm' not in s:
                    lista_superficie.append(None)
                else:
                    lista_superficie.append(re.findall(r'\d+', s)[0])
            else:
                lista_superficie.append(None) 
            
            #ascensor
            if keyword in vivienda_str:
                lista_ascensor.append('SI')

            else:
                lista_ascensor.append('NO') 
                
            #planta
            
            if planta in vivienda_str:
                planta_pos=re.search(r'(Planta)',vivienda_str)
                planta_pos_text=planta_pos.start()
                p=vivienda_str[planta_pos_text-3:planta_pos_text-2]
                lista_planta.append(p)        

            elif planta_baja in vivienda_str:
                planta_pos_baja=re.search(r'(planta)',vivienda_str)
                planta_pos_text_baja=planta_pos_baja.start()
                lista_planta.append(0)        

            else:
                lista_planta.append(None)
            
            #link
            links = vivienda.find('a',href=True)
            href = links['href']
            fotocasa = 'https://www.fotocasa.es/'
            if '/es/comprar/vivienda/' in href:
                lista_links.append(fotocasa+href)
            
            
            #fecha
            
            fecha=vivienda.find('span',class_='re-CardTimeAgo').getText()
            if re.search(r'\d+', fecha):
                fecha_int=int(re.search(r'\d+', fecha).group())
            else:
                fecha_int=0
            lista_fechas.append(fecha_int)
            
                
        except AttributeError:
            pass
    
    return lista_precios, lista_titulos,lista_habitaciones,lista_baños,lista_superficie,lista_ascensor,lista_planta,lista_links,lista_fechas

In [22]:
#df=get_data(viviendas)


In [23]:
def get_max_value(driver):
    resultSet = driver.find_element(By.XPATH,"//ul[@class='sui-MoleculePagination']")
    list_pages=[]
    all_li = resultSet.find_elements(By.TAG_NAME,"li")
    for li in all_li:
        text = li.text
        try:
            text=int(text)
            list_pages.append(text)
        except:
            pass
        
    max_value=max(list_pages)
    return max_value

In [24]:
#max_value=get_max_value(driver)

In [25]:
def next_page(driver):
    try:
        time.sleep(3)
        buttons = driver.find_element(By.XPATH,"//ul/li[@class='sui-MoleculePagination-item'][last()]/a[@class='sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded']")
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(buttons)).click()
        time.sleep(7)
    except:
        time.sleep(3)
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='ab-close-button']"))).click()
        time.sleep(3)
        buttons = driver.find_element(By.XPATH,"//ul/li[@class='sui-MoleculePagination-item'][last()]/a[@class='sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded']")
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(buttons)).click()
        time.sleep(7)

In [26]:
#next_page(driver)

In [27]:
#scroll_down(driver)

In [28]:
#viviendas=get_html(driver)

In [29]:
#n_df=get_data(viviendas)

In [30]:
#df = pd.concat([df,n_df],axis=0)

In [31]:
#df.shape

In [32]:
def batch(distrito):
    driver = initiate()
    buscar_distrito(distrito,driver)
    scroll_down(driver)
    max_value=get_max_value(driver)
    viviendas = get_html(driver)
    df=get_data(viviendas,distrito)
    for i in range(max_value-1):
        next_page(driver)
        scroll_down(driver)
        viviendas=get_html(driver)
        n_df=get_data(viviendas,distrito)
        print(n_df.shape)
        df = pd.concat([df,n_df],axis=0)
        print(df.shape)
        
    return df


In [48]:
if __name__ == '__main__':
    
    busquedas = ['Carolinas Bajas Alicante','San Mames - La Palomera Leon','Chantría la lastra Leon','San Esteban Leon Capital', 'Campoamor Alicante','Pla del Bon Repos - La Goteta Alicante']
    df = pd.DataFrame()
    for busqueda in busquedas:
        data = batch(busqueda)
        df = pd.concat([df, data],axis=0)
        print(df.shape)


Mean length value of the html: 6301.129411764706


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".sui-AtomButton.sui-AtomButton--primary.sui-AtomButton--outline.sui-AtomButton--center.sui-AtomButton--small.sui-AtomButton--link.sui-AtomButton--empty.sui-AtomButton--rounded"}
  (Session info: chrome=110.0.5481.77)
Stacktrace:
0   8b7cb5f8305c528d_chromedriver       0x000000010ccd0138 8b7cb5f8305c528d_chromedriver + 4923704
1   8b7cb5f8305c528d_chromedriver       0x000000010cc489d3 8b7cb5f8305c528d_chromedriver + 4368851
2   8b7cb5f8305c528d_chromedriver       0x000000010c891787 8b7cb5f8305c528d_chromedriver + 472967
3   8b7cb5f8305c528d_chromedriver       0x000000010c8d5ba3 8b7cb5f8305c528d_chromedriver + 752547
4   8b7cb5f8305c528d_chromedriver       0x000000010c8d5df1 8b7cb5f8305c528d_chromedriver + 753137
5   8b7cb5f8305c528d_chromedriver       0x000000010c91c734 8b7cb5f8305c528d_chromedriver + 1042228
6   8b7cb5f8305c528d_chromedriver       0x000000010c8fefdd 8b7cb5f8

In [43]:
df

,Título,Precio,Habitaciones,Planta,Superficie,Baños,Ascensor,Fecha Publicación,Links,Barrio,Ciudad
0,Piso con ascensor en La Chantría - La Lastra,79.000 €,3,None,85,1,SI,0,https://www.fotocasa.es//es/comprar/vivienda/l...,Chantría la lastra Leon,Leon
1,Piso con terraza en La Chantría - La Lastra,58.000 €,3,3,93,1,SI,0,https://www.fotocasa.es//es/comprar/vivienda/l...,Chantría la lastra Leon,Leon
2,"Piso en Calle Dos Hermanas 14, La Chantría - ...",350.000 €,4,None,140,2,SI,3,https://www.fotocasa.es//es/comprar/vivienda/l...,Chantría la lastra Leon,Leon
3,"Piso en Alcalde Miguel Castaño, 18, La Chantrí...",225.000 €,5,1,135,2,SI,37,https://www.fotocasa.es//es/comprar/vivienda/l...,Chantría la lastra Leon,Leon
4,"Piso con terraza en Alcalde Miguel Castaño, La...",180.000 €,3,3,118,2,SI,30,https://www.fotocasa.es//es/comprar/vivienda/l...,Chantría la lastra Leon,Leon
...,...,...,...,...,...,...,...,...,...,...,...
2,Casa o chalet con 1 baño en San Esteban,62.000 €,3,None,107,1,NO,22,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
3,Piso con 1 baño en San Esteban,48.400 €,2,None,80,1,NO,170,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
4,"Piso en Calle Diecinueve de Octubre, 2, San Es...",225.000 €,2,None,85,2,SI,201,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon
5,"Piso en Plaza Mariano Andres 147, San Esteban",18.000 €,2,1,55,1,NO,55,https://www.fotocasa.es//es/comprar/vivienda/l...,San Esteban Leon Capital,Leon


In [44]:
df.to_csv('for_sale.csv')